In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
data= pd.read_csv("2010to2017nsWithoutNA.csv")

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
 
split = StratifiedShuffleSplit(n_splits = 1,test_size = 0.2,random_state = 42)
 
#根据mnist["target"]来进行分层采样
for train_index,test_index in split.split(data,data.iloc[:,-1]):
    train_set = data.iloc[train_index,:]
    test_set = data.iloc[test_index,:]
 
print(len(train_set),len(test_set))

In [ ]:
#because SDV and TABNet has conflict we need to get this data advance
new_data = pd.read_csv('nsGAN.csv')
train_set1=train_set.append(new_data)

In [ ]:
def getPar(model,dist,data,niter):
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import RandomizedSearchCV
    x,y=data.shape

    clf1 = model

    param_dist = dist
    grid = RandomizedSearchCV(clf1,param_dist,cv = 3,scoring = "balanced_accuracy",n_iter=niter,n_jobs = -1)

    #train
    grid.fit(data.iloc[:,0:y-1],data.iloc[:,y-1])
    #get best parameter
    print(grid.best_score_)
    return grid.best_params_

In [ ]:
#from xgboost import XGBClassifier
from pyod.models.xgbod import XGBOD

clf = XGBOD()
#clf = XGBClassifier()
dic = {
        'n_estimators':range(80,2000,4),
        'max_depth':range(2,15,1),
        'learning_rate':np.linspace(0.01,2,20),
        'subsample':np.linspace(0.7,0.9,20),
        'colsample_bytree':np.linspace(0.5,0.98,10)
        }
para9=getPar(clf,dic,test_set,1)

my_model = XGBOD(n_estimators=para9['n_estimators'], learning_rate=para9['learning_rate'],
                     max_depth=para9['max_depth'],subsample=para9['subsample'],
                     colsample_bytree=para9['colsample_bytree'],
                     n_jobs=-1)
my_model.fit(train_set1.iloc[:,0:8],train_set1.iloc[:,8])

pred=my_model.predict(test_set.iloc[:,0:8])

In [ ]:
y_tru1=test_set.iloc[:,8]
sns.set()
f,ax=plt.subplots()
C2= confusion_matrix(y_tru1, pred, labels=[0, 1])
print(C2) #打印出来看看
sns.heatmap(C2,annot=True,ax=ax) #画热力图

ax.set_title('confusion matrix') #标题
ax.set_xlabel('predict') #x轴
ax.set_ylabel('true') #y轴

TABNet

In [ ]:
x=train_set1.iloc[:,0:8].to_numpy()
y=train_set1.iloc[:,8].to_numpy()
a=test_set.iloc[:,0:8].to_numpy()
b=test_set.iloc[:,8].to_numpy()
c=test_set.iloc[:,0:8].to_numpy()

In [ ]:
#we need to try some combinations of these parameters
hyperparameter_search_space = {
    "n_d": [8, 16, 24, 32, 64, 128], # n_a=n_d
    "n_steps": [3, 4, 5, 6, 7, 8, 9, 10],
    "gamma": [1.0, 1.2, 1.5, 2.0],
    "lambda_sparse": [0, 0.000001, 0.0001, 0.001, 0.01, 0.1],
    "momentum": [0.6, 0.7, 0.8, 0.9, 0.95, 0.98]
}

In [ ]:
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

clf = TabNetClassifier(n_d=8,n_a=8,n_steps=8,gamma=1.5,lambda_sparse=0.001,momentum=0.8)  #TabNetRegressor()
clf.fit(
  x,y,
  eval_set=[(a,b)],eval_metric=['balanced_accuracy'],batch_size=1000
)
preds = clf.predict(c)

In [ ]:
y_tru1=test_set.iloc[:,8]
sns.set()
f,ax=plt.subplots()
C2= confusion_matrix(y_tru1, preds, labels=[0, 1])
print(C2) #打印出来看看
sns.heatmap(C2,annot=True,ax=ax) #画热力图

ax.set_title('confusion matrix') #标题
ax.set_xlabel('predict') #x轴
ax.set_ylabel('true') #y轴